In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [2]:
path_df = r"C:\ICE-1\ICE-1\02 Exploratory Data Analysis\News_dataset.pickle"

with open(path_df, 'rb') as data:
    df = pickle.load(data)

In [3]:
df.head()

,Article Content,Category,id,News_length
0,The term ‘autopilot’ has been used for decades...,Autonomous Cars,1,425.0
1,Tesla most famously branded its self-driving f...,Autonomous Cars,1,220.0
2,This article provides a breakdown of cars with...,Autonomous Cars,1,151.0
3,"First, let’s define exactly what constitutes ‘...",Autonomous Cars,1,354.0
4,Learn more about this by reading our article o...,Autonomous Cars,1,105.0


In [4]:
df.loc[5]['Article Content']

'Autopilot is a generic term used in multiple industries, particularly aircraft, and generally means that something can guide itself without human intervention.  In relation to cars in 2021, most mainstream carmakers are focused on, Level 2 autonomy. This level allows the vehicle to take over most steering, acceleration and braking functions, but still requires that the driver remain fully attentive to the driving situation and be able to intervene at any moment. It is not driverless, fully autonomous driving, like robotaxis from Waymo or Cruise (that are now testing in California).'

In [5]:
df['Content_Parsed_1'] = df['Article Content'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")

In [6]:
text = "Mr Greenspan\'s"
text

"Mr Greenspan's"

In [7]:
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')

In [8]:
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

In [9]:
punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

<ipython-input-9-cdceb98d92a9>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')


In [10]:
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

In [11]:
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

------------------------------------------------------------


[nltk_data] Error loading punkt: <urlopen error [WinError 10065] A
[nltk_data]     socket operation was attempted to an unreachable host>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [12]:
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()

In [13]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = str(text).split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [14]:
df['Content_Parsed_5'] = lemmatized_text_list

In [15]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [16]:
stop_words = list(stopwords.words('english'))

In [17]:
stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [18]:
example = "me eating a meal"
word = "me"

# The regular expression is:
regex = r"\b" + word + r"\b"  # we need to build it like that to work properly

re.sub(regex, "StopWord", example)

'StopWord eating a meal'

In [19]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

<ipython-input-19-3b7196a1b53b>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')


In [20]:
df.loc[5]['Article Content']

'Autopilot is a generic term used in multiple industries, particularly aircraft, and generally means that something can guide itself without human intervention.  In relation to cars in 2021, most mainstream carmakers are focused on, Level 2 autonomy. This level allows the vehicle to take over most steering, acceleration and braking functions, but still requires that the driver remain fully attentive to the driving situation and be able to intervene at any moment. It is not driverless, fully autonomous driving, like robotaxis from Waymo or Cruise (that are now testing in California).'

In [21]:
df.loc[5]['Content_Parsed_1']

'Autopilot is a generic term used in multiple industries, particularly aircraft, and generally means that something can guide itself without human intervention.  In relation to cars in 2021, most mainstream carmakers are focused on, Level 2 autonomy. This level allows the vehicle to take over most steering, acceleration and braking functions, but still requires that the driver remain fully attentive to the driving situation and be able to intervene at any moment. It is not driverless, fully autonomous driving, like robotaxis from Waymo or Cruise (that are now testing in California).'

In [22]:
df.loc[5]['Content_Parsed_2']

'autopilot is a generic term used in multiple industries, particularly aircraft, and generally means that something can guide itself without human intervention.  in relation to cars in 2021, most mainstream carmakers are focused on, level 2 autonomy. this level allows the vehicle to take over most steering, acceleration and braking functions, but still requires that the driver remain fully attentive to the driving situation and be able to intervene at any moment. it is not driverless, fully autonomous driving, like robotaxis from waymo or cruise (that are now testing in california).'

In [23]:
df.loc[5]['Content_Parsed_3']

'autopilot is a generic term used in multiple industries particularly aircraft and generally means that something can guide itself without human intervention  in relation to cars in 2021 most mainstream carmakers are focused on level 2 autonomy this level allows the vehicle to take over most steering acceleration and braking functions but still requires that the driver remain fully attentive to the driving situation and be able to intervene at any moment it is not driverless fully autonomous driving like robotaxis from waymo or cruise (that are now testing in california)'

In [24]:
df.loc[5]['Content_Parsed_4']

'autopilot is a generic term used in multiple industries particularly aircraft and generally means that something can guide itself without human intervention  in relation to cars in 2021 most mainstream carmakers are focused on level 2 autonomy this level allows the vehicle to take over most steering acceleration and braking functions but still requires that the driver remain fully attentive to the driving situation and be able to intervene at any moment it is not driverless fully autonomous driving like robotaxis from waymo or cruise (that are now testing in california)'

In [25]:
df.loc[5]['Content_Parsed_5']

'autopilot be a generic term use in multiple industries particularly aircraft and generally mean that something can guide itself without human intervention  in relation to cars in 2021 most mainstream carmakers be focus on level 2 autonomy this level allow the vehicle to take over most steer acceleration and brake function but still require that the driver remain fully attentive to the drive situation and be able to intervene at any moment it be not driverless fully autonomous drive like robotaxis from waymo or cruise (that be now test in california)'

In [26]:
df.loc[5]['Content_Parsed_6']

'autopilot   generic term use  multiple industries particularly aircraft  generally mean  something  guide  without human intervention   relation  cars  2021  mainstream carmakers  focus  level 2 autonomy  level allow  vehicle  take   steer acceleration  brake function  still require   driver remain fully attentive   drive situation   able  intervene   moment    driverless fully autonomous drive like robotaxis  waymo  cruise (   test  california)'

In [27]:
df.head(1)

,Article Content,Category,id,News_length,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,The term ‘autopilot’ has been used for decades...,Autonomous Cars,1,425.0,The term ‘autopilot’ has been used for decades...,the term ‘autopilot’ has been used for decades...,the term ‘autopilot’ has been used for decades...,the term ‘autopilot’ has been used for decades...,the term ‘autopilot’ have be use for decades w...,term ‘autopilot’ use decades within aircr...


In [28]:
list_columns = ["Category", "Article Content", "Content_Parsed_6"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

In [29]:
df.head()

,Category,Article Content,Content_Parsed
0,Autonomous Cars,The term ‘autopilot’ has been used for decades...,term ‘autopilot’ use decades within aircr...
1,Autonomous Cars,Tesla most famously branded its self-driving f...,tesla famously brand self-driving feature “...
2,Autonomous Cars,This article provides a breakdown of cars with...,article provide breakdown cars autopilot ...
3,Autonomous Cars,"First, let’s define exactly what constitutes ‘...",first let’ define exactly constitute ‘autopil...
4,Autonomous Cars,Learn more about this by reading our article o...,learn read article sae self-driving leve...


In [30]:
category_codes = {
    'Autonomous Cars': 0,
    'Bill Resource': 1,
    'Companies Working on Autonomous Cars': 2
}

In [31]:
df['Category_Code'] = df['Category']
df = df.replace({'Category_Code':category_codes})

In [32]:
df.head()

,Category,Article Content,Content_Parsed,Category_Code
0,Autonomous Cars,The term ‘autopilot’ has been used for decades...,term ‘autopilot’ use decades within aircr...,0
1,Autonomous Cars,Tesla most famously branded its self-driving f...,tesla famously brand self-driving feature “...,0
2,Autonomous Cars,This article provides a breakdown of cars with...,article provide breakdown cars autopilot ...,0
3,Autonomous Cars,"First, let’s define exactly what constitutes ‘...",first let’ define exactly constitute ‘autopil...,0
4,Autonomous Cars,Learn more about this by reading our article o...,learn read article sae self-driving leve...,0


In [33]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

In [34]:
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [35]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(926, 300)
(164, 300)


In [36]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'Autonomous Cars' category:
  . Most correlated unigrams:
. lane
. assist
. core
. feature
. autopilot
  . Most correlated bigrams:
. motor vehicle
. major developments

# 'Bill Resource' category:
  . Most correlated unigrams:
. nan
. vehicle
. 28
. person
. section
  . Most correlated bigrams:
. motor vehicle
. self driving

# 'Companies Working on Autonomous Cars' category:
  . Most correlated unigrams:
. closer
. company
. developments
. major
. nan
  . Most correlated bigrams:
. closer look
. major developments



In [37]:
bigrams

['autonomous vehicles',
 'automate drive',
 'without human',
 'driver assistance',
 'drive system',
 'autonomous vehicle',
 'child restraint',
 'subsection section',
 'pursuant section',
 'driving technology',
 'driver license',
 'requirements section',
 'human driver',
 'vehicle operate',
 'statutes amend',
 'amend read',
 'arizona revise',
 'revise statutes',
 'fully autonomous',
 'company also',
 'driving cars',
 'section 28',
 'autonomous drive',
 'motor vehicle',
 'self driving',
 'closer look',
 'major developments']

In [39]:
# X_train
with open('X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)